### Imports

In [11]:
import pandas as pd
import numpy as np
import math

### Load N-Gram Master Vocab CSV

###### This CSV / Data currently contains the master vocabulary of N-Grams (unique set) and their frequencies in each language. 

In [12]:
df = pd.read_csv("../data/vocab/master_vocab_char3_ngrams.csv")
df = df.rename(columns={"Unnamed: 0": "ngram"})
ngrams = df["ngram"]

### Feature Engineering

###### Right now, the current feature only represents the **raw frequencies** of how many times each n-gram appears in a given language. This can **skew language similarity**, since common n-grams that appear across all languages will dominate the comparison. For example, languages might be considered similar if N-Grams that contain common vowels such as "_ea" or _ou" have high raw frequencies for both, even when in reality they are not similar languages.  To address this, we will apply **TF-IDF**: a weighting method that adjusts n-gram importance based on how **unique** and **informative** each one is to a specific language.


#### TF (Term Frequency)

###### This is the amount of times the N-Gram appears in the document per language. Since each of our documents vary in size, we will have to normalize to account for this by dividing by the total number of terms per document.

In [13]:
df_numeric = df.drop(columns=["ngram"])
tf_numeric = df_numeric.div(df_numeric.sum(axis=0), axis=1)
tf = pd.concat([ngrams, tf_numeric], axis=1)

In [14]:
tf.head()

,ngram,adasen,bikolano,cebuano,chavacano,english,ilokano,ilonggo,kinaray-a,masbatenyo,paranan,romblomanon,spanish,tagalog,tausug,waray,yami
0,gol,0.000002,0.000039,0.000042,0.000009,0.000022,0.000023,0.000030,0.000019,0.000021,0.000005,0.000005,0.000055,0.000093,0.000000,0.000089,0.000032
1,tig,0.000022,0.000041,0.000326,0.000240,0.000000,0.000014,0.000175,0.000047,0.000212,0.000054,0.000123,0.000117,0.000040,0.000037,0.000267,0.000000
2,sej,0.000000,0.000000,0.000000,0.000021,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000007,0.000000,0.000000,0.000000,0.000000
3,ubd,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000002,0.000003,0.000000
4,tka,0.000018,0.000000,0.000000,0.000000,0.000000,0.000138,0.000000,0.000000,0.000000,0.000202,0.000000,0.000000,0.000000,0.000000,0.000003,0.000083


### DF (Document Frequency)

###### This is how many documents/languages the N-Gram appears in.

In [15]:
df_numeric = df.drop(columns=["ngram"])
df_count = (df_numeric > 0).sum(axis=1)
df_full = pd.DataFrame({"ngram": df["ngram"], "df": df_count})

In [16]:
df_full.head()

,ngram,df
0,gol,15
1,tig,14
2,sej,2
3,ubd,2
4,tka,5


### IDF (Inverse Document Frequency)

###### This gets how unique a document is: **log(N/DF)**, where N = total amount of documents / languages.

In [17]:
N = df_numeric.shape[1]
idf = np.log(N / (df_count + 1e-10)) # we add +1e-10 to avoid division by zero, although with how the data is processed, this should never happen
idf_full = pd.DataFrame({"ngram": df["ngram"], "idf": idf})

### TF-IDF (Term Frequency - Inverse Document Frequency)

###### Now we multiply TF by IDF to get the final TF-IDF scores for each n-gram in each language.

In [18]:
# multiply TF by IDF for each language
tfidf_numeric = tf_numeric.multiply(idf.values, axis=0)
tfidf = pd.concat([ngrams, tfidf_numeric], axis=1)

In [19]:
tfidf.head()

,ngram,adasen,bikolano,cebuano,chavacano,english,ilokano,ilonggo,kinaray-a,masbatenyo,paranan,romblomanon,spanish,tagalog,tausug,waray,yami
0,gol,1.297854e-07,0.000002,0.000003,5.534609e-07,0.000001,0.000001,0.000002,0.000001,0.000001,3.344294e-07,3.175897e-07,0.000004,0.000006,0.000000,0.000006,0.000002
1,tig,2.953813e-06,0.000006,0.000044,3.206338e-05,0.000000,0.000002,0.000023,0.000006,0.000028,7.265377e-06,1.642748e-05,0.000016,0.000005,0.000005,0.000036,0.000000
2,sej,0.000000e+00,0.000000,0.000000,4.458150e-05,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000014,0.000000,0.000000,0.000000,0.000000
3,ubd,0.000000e+00,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000004,0.000005,0.000000
4,tka,2.105161e-05,0.000000,0.000000,0.000000e+00,0.000000,0.000160,0.000000,0.000000,0.000000,2.350640e-04,0.000000e+00,0.000000,0.000000,0.000000,0.000003,0.000097


In [20]:
tfidf.to_csv("../results/tfidf.csv", index=False)